The file format is as follows. In each instance, the number of variables and the number of clauses is the same, and this number is specified on the first line of the file. Each subsequent line specifies a clause via its two literals, with a number denoting the variable and a "-" sign denoting logical "not". For example, the second line of the first data file is "-16808 75250", which indicates the clause ¬x16808∨x75250.

In [3]:
## Get the clauses into the notebook:
def importClauses(fileNumber):
    with open('Downloads/algo2_2sat' + str(fileNumber) + '.txt') as f:
        n = int(f.readline().strip('\n'))
        clauses = [(int(c[0]), int(c[1])) for c in [line.strip('\n').split() for line in f.readlines()]]
    return n, clauses

In [4]:
n1, clauses1 = importClauses(1)

In [5]:
n1

100000

In [6]:
clauses1[12]

(-80980, -79150)

In [7]:
len(clauses1)

100000

In [8]:
lits = []
for c in clauses1:
    lits.append(abs(c[0]))
    lits.append(abs(c[1]))

In [9]:
len(lits)

200000

In [10]:
print(max(lits), min(lits))  ## So variables are numbered from 1 to n

100000 1


## First try a quick Papadimitriou on the smallest problem, to see how long it takes

In [174]:
def papa(clauseList):
    """
    Assuming an input of a list of clause tuples, with labels from 1 to n consecutive, and minus signs for negation
    """
    import random   
    
    n = len(clauseList)
    ## Initialize all positive variables to False and negative variables to True, and pad with a 0 up front for 1-labeling
    truths = [False for _ in range(n+1)] + [True for _ in range(n)] 
    ## Remove clauses that can be automatically and permanently satisfied, to speed things up
    freebies = sat(clauseList)
    notFree = []
    for c in clauseList:
        if c[0] not in freebies and c[1] not in freebies:
            notFree.append(c)
    m = len(notFree)
    print("number of not free clauses: " + str(m))
    for _ in range(int(log2(m))):   ## how many times to try the main routine before giving up
        for _ in range(2*m**2):   ## note: 200 billion, even for the smallest problem here, unless freebies are optimized
            u = unsat(notFree, m, truths)
            if u == -1:   ## if all clauses are satisfied, return True
                return True
            var = random.choice(notFree[u])  ## pick a variable from the unsat clause to flip
            truths[var] = not truths[var]
            truths[-var] = not truths[-var]
            
    return False

In [90]:
def unsat(clauses, numclauses, truths):
    ## pick a random starting point
    r = randint(0, numclauses-1)
    ## first part of the list to test
    for i in range(r, numclauses):
        if truths[clauses[i][0]] or truths[clauses[i][1]]: continue
        return i
    for i in range(r):
        if truths[clauses[i][0]] or truths[clauses[i][1]]: continue
        return i
    return -1


In [170]:
unsat([(1,-2), (2,-3), (3,-4), (4,1)], 4, [False, False, False, False, False, True, True, True, True])

3

In [175]:
papa([(1,-2), (2,-3), (3,-2), (4,3)])

number of not free clauses: 2


True

In [153]:
papa(clauses1)  ## That took over a minute, even for just 100,000 inner loops, and it returned True, so who knows how many in reality?

39821


True

In [154]:
n2, clauses2 = importClauses(2)

In [155]:
n2

200000

In [157]:
papa(clauses2)  ## takes too long with my barebones papa, but maybe with the noDupes optimization....

In [158]:
n3, clauses3 = importClauses(3)

In [159]:
n3

400000

In [161]:
papa(clauses3)  ## takes too long

Maybe a couple of optimizations?

In [135]:
## See which variables are set to same truth in both of their clauses, and init them to that truth
def findDupes(cList):
    seen = set()
    dupes = []
    for c in cList:
        for v in c:
            if v in seen:
                dupes.append(v)
            else:
                seen.add(v)
    return dupes
    

In [146]:
def sat(cList):
    free = set()  ## keep vars here until they get negated
    negated = set()  ## permanently negated once you enter here
    for c in cList:
        for v in c:
            if abs(v) not in negated:  ## just keep abs val of variable name 
                if -v in free:
                    free.remove(-v)
                    negated.add(abs(v))
                else:
                    free.add(v)
    return free
               

In [147]:
free = sat(clauses2)

In [149]:
len(free)

92883

In [176]:
papa(clauses1)  ## with optimization, time was cut from 88 secs to 24 secs

number of not free clauses: 39821


True

In [178]:
papa(clauses2)  # still too slow

Still too slow--Maybe remove all clauses with permanently satisfiable literals in them to avoid checking if they're SAT yet, since they always will be.

In [109]:
n4, clauses4 = importClauses(4)

In [162]:
n4

600000

In [164]:
papa(clauses4) # too long

In [129]:
n5, clauses5 = importClauses(5)

In [130]:
n5

800000

In [165]:
papa(clauses5)  ## too long

In [132]:
n6, clauses6 = importClauses(6)

In [168]:
papa(clauses6)  ## too long

In [179]:
n6  

1000000

## OK, so Papa is too slow, as implemented, for even the second smallest set, even with removal of half the clauses.   Time to try strongly connected components.

Build a graph from the clauselist, with directed edges going from the negation of one of the variables to the positive of the other, for each pair.  So e.g. (-19, 28), meaning (not 19 || 28), will give rise to 2 directed edges:
19 --> 28 (since 19 being True implies 28 is True) and -28 --> -19 (since 28 being False implies 19 is False).  The reasoning is that any one 'truth vertex' can be seen as implying all the other vertices in its strongly connected component.  Therefore, once the components are determined, they can be scanned to see if any vertex's negation is strongly connected to it, which would imply the clauses are unsatisfiable.

In [184]:
def graphs(clauseList):
    graphDict = dict()
    reverseDict = dict()
    for c in clauseList:
        if -c[0] in graphDict:   ## NOT c[0] implies c[1]
            graphDict[-c[0]].append(c[1])
        else:
            graphDict[-c[0]] = [c[1]]
        if -c[1] in graphDict:    ## NOT c[1] implies c[0]
            graphDict[-c[1]].append(c[0])
        else:
            graphDict[-c[1]] = [c[0]]
        if c[1] in reverseDict:
            reverseDict[c[1]].append(-c[0])
        else:
            reverseDict[c[1]] = [-c[0]]
        if c[0] in reverseDict:
            reverseDict[c[0]].append(-c[1])
        else:
            reverseDict[c[0]] = [-c[1]]
            
    return graphDict, reverseDict

In [187]:
def reverseDFS(reverseDict, maxNode):
    
    newOrder = []  # build the crucial ordering during the first DFS pass
    ## keep track of explored nodes, with +1 for one-indexing of graph, and 2*maxNode because of negative labeled nodes
    seen = [False for _ in range(2*maxNode+1)]  
    
    def dfs(source):
        try:
            for dest in reverseDict[source]:
                if not seen[dest]:
                    seen[dest] = True
                    dfs(dest)
            newOrder.insert(0,source)
        except: KeyError
            
    for node in reverseDict.keys():
        if not seen[node]:
            seen[node] = True
            dfs(node)
     
    return newOrder

In [188]:
def forwardDFS(graphDict, maxNode, order):
    
    reps = {}  # store a representative node for each SCC found during the second DFS pass
    # keep track of explored nodes, with +1 for one-indexing of graph, and 2*maxNode because of negative labeled nodes
    seen = [False for _ in range(2*maxNode+1)]  
    
    def dfs(source, rep):
        try:
            for dest in graphDict[source]:
                if seen[dest]: continue
                seen[dest] = True
                reps[rep].add(dest)
                dfs(dest, rep)
        except: KeyError

    for node in order:
        if seen[node]: continue
        seen[node] = True
        reps[node] = {node}
        dfs(node, node)
        
    return reps

In [212]:
def scc(clauseList):
    """
    Assuming an input of a list of clause tuples, with labels from 1 to n consecutive, and minus signs for negation
    """
    import random   
    
    n = len(clauseList)
    ## Remove clauses that can be automatically and permanently satisfied, to speed things up
    freebies = sat(clauseList)
    notFree = []
    for c in clauseList:
        if c[0] not in freebies and c[1] not in freebies:
            notFree.append(c)
    m = len(notFree)
    print("number of not free clauses: " + str(m))

    adjDict, revDict = graphs(clauseList)
    hiNode = max(abs(k) for k in adjDict.keys())
   
    ordering = reverseDFS(revDict, hiNode)
    strongs = forwardDFS(adjDict, hiNode, ordering)
    
    return strongs



In [190]:
import sys, threading
sys.setrecursionlimit(800000)
threading.stack_size(67108864)

thread = threading.Thread()
thread.start()


In [199]:
components = list(strongs2.values())

components.sort(key=lambda x: len(x), reverse=True)

[len(comp) for comp in components[:10]]

[46, 46, 8, 4, 4, 1, 1, 1, 1, 1]

In [205]:
def check(sccDict):
    for k in sccDict:
        if -k in sccDict[k]:
            print(k, sccDict[k])
            return False
        for v in sccDict[k]:
            if -v in sccDict[k]:
                print(k, sccDict[k])
                return False
    return True

In [214]:
strongs1 = scc(clauses1)   # ~8 secs
check(strongs1)

number of not free clauses: 39821


True

In [215]:
strongs2 = scc(clauses2)   ## ~25 secs  for 80K clauses

number of not free clauses: 80186


In [201]:
check(strongs2)

False

In [216]:
strongs3 = scc(clauses3)   ## ~107 secs
check(strongs3)

number of not free clauses: 158903


True

In [217]:
strongs4 = scc(clauses4)
check(strongs4)

number of not free clauses: 239864


True

In [218]:
strongs5 = scc(clauses5)
check(strongs5)

number of not free clauses: 320834
618692 {223530, 618692, -223530, -618692}


False

In [219]:
strongs6 = scc(clauses6)   # 14-15 mins
check(strongs6)

number of not free clauses: 398526
273101 {976968, -410420, 273101, -273101, 410420, -976968}


False

### Someone in the discussion forum pointed out that you can recurse on the optimized set of clauses to drastically reduce the problem size in the input:

In [220]:
def prune(cList):
    n = len(cList)
    freebies = sat(cList)
    notFree = []
    for c in cList:
        if c[0] not in freebies and c[1] not in freebies:
            notFree.append(c)
    m = len(notFree)
    if m == n:
        return cList
    else:
        return prune(notFree)

In [221]:
pruned1 = prune(clauses1)

In [222]:
len(pruned1)

6

In [223]:
pruned1

[(95476, -15789),
 (-98193, 79715),
 (58504, 98193),
 (-95476, 73096),
 (-79715, -73096),
 (15789, -58504)]

In [224]:
check(scc(pruned1))

number of not free clauses: 6


True

In [225]:
check(scc(prune(clauses2)))

number of not free clauses: 57
134592 {134592, -134592, 9187, 76873, -162741, 162741, -76873, -9187}


False

In [226]:
check(scc(prune(clauses3)))

number of not free clauses: 295


True

In [227]:
check(scc(prune(clauses4)))

number of not free clauses: 11


True

In [228]:
check(scc(prune(clauses5)))

number of not free clauses: 101
223530 {-618692, 223530, 618692, -223530}


False

In [229]:
check(scc(prune(clauses6)))

number of not free clauses: 26
273101 {976968, -410420, 273101, -273101, 410420, -976968}


False

Well, that certainly makes it easier. 